# Lab 3
Convolutional Neural Networks

## Useful links:

Information on Pytorch layers: [link](https://pytorch.org/docs/stable/nn.html)

And more specifically:


*   Linear layers: [link](https://pytorch.org/docs/stable/nn.html#linear-layers)
*   Loss layers: [link](https://pytorch.org/docs/stable/nn.html#loss-functions)
*   Activation functions: [link](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
*   Datasets and dataloaders: [link](https://pytorch.org/docs/stable/data.html)
*   Saving and loading models: [link](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

TSNE visualization: [link](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

#Information about network training:

### How to use loss function:
```
# define the loss function once (before training):
loss_function = nn.CrossEntropyLoss()

# to calculate our loss after the forward pass:
current_loss = loss_function(outputs, target)

# perform backward pass:
current_loss.backward()
```

### Network optimization (learning):
```
# define the optimizer once (before training):
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # optimizing parameters (weights) with learning rate 0.01

# before performing the backward pass clear the information about the gradients from the previous pass:
optimizer.zero_grad()

# after performing the backward pass
optimizer.step()
```

### Forward pass:
```
#if you have already defined a model, the only thing you have to do is:
outputs = model(inputs)
```

#Information about defining the network:

Every model will have similar structure:
```
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # Define your layers here

    def forward(self, x):
        # Define the forward pass operations here
        return x
```

There are two distinc functions here, the ```__init__``` and ```forward```.

In the ```__init__``` function we will define layers that we will later on use in our forward pass.

In the ```forward``` function we will define step by step what should happen in our forward pass.

# Layers
### Linear
First layer we can use is a linear (or fully connected) layer. We can define it as:
```
# Linear layer with 5 inputs and 2 outputs (goes inside the __init__)
self.fc = torch.nn.Linear(in_features=5, out_features=2)

# moving data through the layer (goes into the forward function)
output = self.fc(input) 
```
### Activation
Activation functions don't have to be defined in the ```__init__``` function as long as they don't have any trainable parameters (and most of them don't have any).
```
# moving data through the layer with sigmoid (goes into the forward function)
output = F.sigmoid(self.fc(input)) 

# moving data through the layer with relu (goes into the forward function)
output = F.relu(self.fc(input)) 
```

### Reshape
Frequently you will have to reshape your input (from 2D to 1D for example).
```
# if input is of shape (N, 10, 10)
output = input.view(-1, 100)
# now output is of shape (N, 100)
```

### 2D Convolution
Definition of the convolution layer
```
# Convolution layer with 10 filters of size 3x3. The input has 5 channels.
self.conv = nn.Conv2d(5, 10, kernel_size=3)

# Forward pass:
output = self.conv(input)
```

### 2D max pooling
```
# Performing a 2D max pooling operation with a kernel of size 2x2
output = F.max_pool2d(self.conv(input), 2)
```

### Dropout
```
# 1D dropout performed on the output of a linear layer
output = F.dropout(self.linear(input))
```

# Data transformations
```
# simplest transformation (transforming image to PyTorch tensor):
transform = transforms.ToTensor()

# you can add more transformations (after you converted image to tensor). Simplest would be normalization (for 1 channel data (grayscale)):
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((mean,), (std,))])
```

# Saving and loading models
Keeping progress of our training is very important. Being able to save and load our previous models will become very helpful.

Working on entire model:
```
PATH = "./mnist_model.pt"
# saving entire model:
torch.save(model, PATH)
# loading entire model:
model = torch.load(PATH)
```

Saving more details. Useful when stopping and resuming training.
```
PATH = "./mnist_model.pt"
# saving more detailed information:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)
# loading more detailed information:
model = Net() # initialize the object first
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # initialize the object first

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
```


# INSTRUCTIONS

## Task 1 - CNN for MNIST

Mount your Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Create a folder in your Google Drive named "data".

You can do it either manually or as command line:
```
%cd /content/gdrive/My\ Drive/
%mkdir data
```

In [3]:
# general path:
data_path = "/content/gdrive/My\ Drive/data/"

Move to that folder:

In [3]:
# go to the folder:
%cd /content/gdrive/My\ Drive/data/
# print out the content of the folder:
%ls

/content/gdrive/My Drive/data
Flowers/  MNIST/


Imports:

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt

Load the dataset.

Modify the transforms for the dataset to include a normalization of data. Numbers for MNIST are: mean is 0.1307, std is 0.3081.

In [8]:
######## Read MNIST ########
# number of subprocesses to use for data loading
num_workers = 0 # means to use all
# how many samples per batch to load
batch_size = 64
# where the dataset is:
dataset_path = "./MNIST"

# convert data to torch.FloatTensor
transform = None # modify it according to the description above. Previously was: transform = transforms.ToTensor()

# create training and test datasets
train_data = datasets.MNIST(root=dataset_path, train=True, download=True, transform=transform)
test_data = datasets.MNIST(root=dataset_path, train=False, download=True, transform=transform)

# create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)

Define the network:

Your Convolutional Neural Network should have the following layers:

2D convolution with 10 kernels of size 5
2D convolution with 20 kernels of size 5
fully connected (linear) layer with output size 50
final fully connected layer with output size 10

In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # Define your layers here

    def forward(self, x):
        # Define the forward pass operations here
        return x

Training the network.

We will iterate through our dataset. For evey iteration we need to:


In [ ]:
# initialize the model:
model = Net()
print(model)

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# specify scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# number of epochs to train the model
n_epochs = 25

# lists to keep track of training progress:
train_loss_progress = []
test_accuracy_progress = []

model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in train_loader:
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(data)
        # calculate the loss
        loss = criterion(outputs, target)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # if you have a learning rate scheduler - perform a its step in here
    scheduler.step()
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    train_loss_progress.append(train_loss)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

    # Run the test pass:
    correct = 0
    total = 0
    model.eval()  # prep model for testing

    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    test_accuracy_progress.append(100 * correct / total)
    print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

Plot the progress using the code below:

In [ ]:
# Plotting:
x_range = np.arange(1, n_epochs+1)
fig, axs = plt.subplots(2)
axs[0].plot(x_range, train_loss_progress, c='b', label="Train loss")
axs[1].plot(x_range, test_accuracy_progress, c='r', label="Test accuracy")
axs[0].legend()
axs[1].legend()
plt.show()

## To do:
Perform following modifications (one at a time) to the forward pass:
1.   Use relu activation functions for both conv layers and for the first linear layer
2.   Add a dropout layer between the two linear layers
3.   Add 2D max pooling layer before performing a relu activation on both conv layers

Note the differences in performances when modifying the network.

In your research report - provide results and analysis of the conducted experiments.



## Task 2 - Flowers dataset

Get the zip file with images from this [link](https://drive.google.com/file/d/1jf6-NFfCRJHIZWrbg-_PT5w-_xmqV_Iv/view?usp=sharing) and put it in your Google Drive in ```/data/Flowers/``` folder.

So the path should look like this:
```
"/content/gdrive/My\ Drive/data/Flowers/flowers.zip"
```

Go to the current directory:

In [4]:
%cd /content/gdrive/My\ Drive/data/Flowers/
%ls

/content/gdrive/My Drive/data/Flowers
flowers.zip  test/  train/  val/


Unzip the file:

In [ ]:
%unzip ./flowers.zip -d ./

Check if the folders are there. You should see 3 folders (train, val and test).

In [ ]:
%ls

Imports and dataloaders:

Implement a following transformations to your data:

For test and validation data:

*   Use a RandomResiedCrop to crop image to size 224x224
*   Perform a random horizontal flip
*   Perform a random rotation by max 10 degrees
*   Perform a color jitter with parameters: brightness=0.4, contrast=0.4, saturation=0.4, hue=0
*   Make it a tensor
*   Normalize it using the following data: means: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]

For test and validation data:

*   Resize to 256x256 img size
*   Crop the center of the image of size 224x224
*   Make it a tensor
*   Normalize it using the following data: means: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import matplotlib.pyplot as plt

batch_size = 64
num_workers = 0

# define transforms:
train_transform = None
test_transform = None

# define datasets:
train_data = datasets.ImageFolder("./train", transform=train_transform)
val_data = datasets.ImageFolder("./val", transform=test_transform)
test_data = datasets.ImageFolder("./test", transform=test_transform)

# define dataloaders:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)

Define your network

List of predefined models: [link](https://pytorch.org/docs/stable/torchvision/models.html)

In [ ]:
model = models.resnet18(pretrained=False)
print(model)

Remove the last layer of the network. Add to it your own layer (with the output dimention corresponding to number of categories in your dataset). In this case it is 102.

In [ ]:
# replace the last fc layer with your own linear layer:

print(model)

Train (and validate):

In [ ]:
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# specify scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# number of epochs to train the model
n_epochs = 20

# lists to keep track of training progress:
train_loss_progress = []
validation_accuracy_progress = []

model.train() # prep model for training

n_iterations = int(len(train_data)/batch_size)

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for iter, (data, target) in enumerate(train_loader):  
        print("Epoch:", epoch, "Iteration:", iter, "out of:", n_iterations)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(data)
        # calculate the loss
        loss = criterion(outputs, target)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update running training loss
        train_loss += loss.item()*data.size(0)
      
    # if you have a learning rate scheduler - perform a its step in here
    scheduler.step()
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

    # Run the test pass:
    correct = 0
    total = 0
    model.eval()  # prep model for validation

    with torch.no_grad():
        for data, target in val_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    print('Accuracy of the network on the validation set: %d %%' % (100 * correct / total))

In [ ]:
# Run the test pass:
correct = 0
total = 0
model.eval()  # prep model for validation

with torch.no_grad():
    for data, target in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy of the network on the validation set: %d %%' % (100 * correct / total))

## To do:

1.   Write a part where you save your model after you're finished training
2.   Train the network above. Provide the training loss, validation accuracy and test accuracy at the end.
3.   Change the network to be pretrained. Decrease the initial learning rate to be 0.001. Train the network this way and provide same performance measures.

Compare the impact the use of a pretrained network had on the final accuracy and on the training in general.